<a href="https://colab.research.google.com/github/vchiang001/sexual_diversity_dissertation/blob/main/Assessing_butts_behaviour_algorithm_performance_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from shapely.geometry import Polygon
from shapely.geometry import Point
import os

In [ ]:
def butts(unpickle, velocity, confidence, side, vel_thresh, coord1, coord2, coord3, coord4, vel_wndw):
    bounding_box_polygon = Polygon([coord1, coord2, coord3, coord4])

    df_vel = pd.read_csv(velocity, index_col=0, header=list(range(3)))
    for col_name in df_vel.columns:
        if "likelihood" in col_name[2]:
            x_col = (col_name[0], col_name[1], 'x')
            y_col = (col_name[0], col_name[1], 'y')
            df_vel.loc[df_vel[col_name] < confidence, [x_col, y_col]] = float('nan')

    unpickle_df = pd.read_csv(unpickle, index_col=0, header=list(range(3)))
    for col_name in unpickle_df.columns:
        if "likelihood" in col_name[2]:
            x_col = (col_name[0], col_name[1], 'x')
            y_col = (col_name[0], col_name[1], 'y')

            unpickle_df.loc[unpickle_df[col_name] < confidence, [x_col, y_col]] = float('nan')

    butts_per_bodypart_df = pd.DataFrame(index=unpickle_df.index, columns=unpickle_df.columns.levels[1])
    prev_x_list = []

    for column in unpickle_df.columns:
        if 'x' in column:
            x_column = column
            y_column = (column[0], column[1], 'y')
            for index, row in unpickle_df.iterrows():
                if bounding_box_polygon.contains(Point(row[x_column], row[y_column])):
                    if prev_x_list:
                        if side == "left":
                            if any(prev_x < row[x_column] for prev_x in prev_x_list):
                                x_vel_column = ('vscc', column[1], 'x')
                                y_vel_column = ('vscc', column[1], 'y')
                                x_vel = df_vel.at[index, x_vel_column]
                                y_vel = df_vel.at[index, y_vel_column]
                                if float(x_vel) > vel_thresh and float(y_vel) > vel_thresh:
                                    butts_per_bodypart_df[column[1]][index] = 1
                        elif side == "right":
                            if any(prev_x > row[x_column] for prev_x in prev_x_list):
                                x_vel_column = ('vscc', column[1], 'x')
                                y_vel_column = ('vscc', column[1], 'y')
                                x_vel = df_vel.at[index, x_vel_column]
                                y_vel = df_vel.at[index, y_vel_column]
                                if float(x_vel) > vel_thresh and float(y_vel) > vel_thresh:
                                    butts_per_bodypart_df[column[1]][index] = 1
                    prev_x_list.append(row[x_column])
                    prev_x_list = prev_x_list[-vel_wndw:]

    butts_df = pd.DataFrame(index=butts_per_bodypart_df.index, columns=['butts'])

    for index, row in butts_per_bodypart_df.iterrows():
        if 1 in row.values:
            butts_df.at[index, 'butts'] = 1
        else:
            butts_df.at[index, 'butts'] = 0

    seconds_df = butts_df.groupby(butts_df.index // 30).any().astype(int)

    return seconds_df

In [ ]:
def roi(file_name, roi_df):
    coords = roi_df[file_name].tolist()
    coord1 = (coords[0], coords[1])
    coord2 = (coords[2], coords[3])
    coord3 = (coords[4], coords[5])
    coord4 = (coords[6], coords[7])
    return coord1, coord2, coord3, coord4

In [ ]:
# Define the folder paths
unpickle_folder_path = "/content/drive/MyDrive/unpickle"
velocity_folder_path = "/content/drive/MyDrive/velocity"
roi_csv_path = "/content/drive/MyDrive/roi.csv"
manual = "/content/drive/MyDrive//manual_butts.csv"
directory_path = '/content/drive/MyDrive/tuning_butts'
confidence = 0.4
vel_thresh = 35
vel_wndw = 35

In [ ]:
roi_df = pd.read_csv(roi_csv_path, index_col=0)
manual = pd.read_csv(manual)
machine_df = pd.DataFrame()

In [ ]:
for filename in os.listdir(unpickle_folder_path):
    file_name = os.path.splitext(filename)[0]
    side = "left" if "L" in file_name else "right"
    unpickle_file_path = os.path.join(unpickle_folder_path, filename)
    velocity_file_path = os.path.join(velocity_folder_path, f"{file_name}.csv")
    coord1, coord2, coord3, coord4 = roi(file_name, roi_df)
    seconds_df = butts(unpickle=unpickle_file_path,
                      velocity=velocity_file_path,
                      confidence=confidence,
                      side=side,
                      vel_thresh=vel_thresh,
                      coord1=coord1,
                      coord2=coord2,
                      coord3=coord3,
                      coord4=coord4,
                      vel_wndw=vel_wndw)
    seconds_df.rename(columns={"butts": file_name}, inplace=True)

    machine_df = pd.concat([machine_df, seconds_df], axis=1)

index_label = f'vel_thresh{vel_thresh}vel_wndw{vel_wndw}'
machine_df = machine_df.apply(pd.to_numeric, errors='coerce')
machine_df = machine_df.fillna(0)
print(machine_df)
manual = manual.apply(pd.to_numeric, errors='coerce')
manual = manual.fillna(0)
print(manual)

machine_df.to_csv(f"{directory_path}/vel_thresh{vel_thresh}vel_wndw{vel_wndw}_machine.csv")

      5R   3R   3L   2R  1R   2L  1L   4R   5L   4L
0    0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0
1    0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0
2    0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0
3    0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0
4    0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0
..   ...  ...  ...  ...  ..  ...  ..  ...  ...  ...
351  0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0
352  0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0
353  0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0
354  0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0
355  0.0  0.0  0.0  0.0   0  0.0   0  0.0  0.0  0.0

[356 rows x 10 columns]
      1L   1R   2L   2R   3L   3R   4L   4R   5L   5R
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4    1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
..   ...  ...  ...  ...  ..

In [ ]:
def calculate_precision(machine, manual, index_label):
    precision_dict = {}
    for column in machine.columns:
        true_positives = ((machine[column] == 1) & (manual[column] == 1)).sum()
        false_positives = ((machine[column] == 1) & (manual[column] == 0)).sum()

        if true_positives + false_positives == 0:
            precision = 0
        else:
            precision = true_positives / (true_positives + false_positives)

        precision_dict[column] = precision

    return pd.DataFrame.from_dict({index_label: precision_dict}, orient='index')


def calculate_recall(machine, manual, index_label):

    recall_dict = {}
    for column in machine.columns:
        true_positives = ((machine[column] == 1) & (manual[column] == 1)).sum()
        false_negatives = ((machine[column] == 0) & (manual[column] == 1)).sum()

        if true_positives + false_negatives == 0:
            recall = 0
        else:
            recall = true_positives / (true_positives + false_negatives)

        recall_dict[column] = recall
    return pd.DataFrame.from_dict({index_label: recall_dict}, orient='index')

In [ ]:
precision_df_all = pd.DataFrame()
recall_df_all = pd.DataFrame()

precision_df = calculate_precision(machine_df, manual, index_label)
print(precision_df)
recall_df = calculate_recall(machine_df, manual, index_label)
print(recall_df)